In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_DNN_FC
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale
#from sklearn.preprocessing import MinMaxScaler, StandardScaler

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [2]:
path='/users/mtaranov/datasets_captureC_blood/'

X_train_node1 = path+'motifs/train_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/train_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/train_out_btw_nodes_3_0.0001/mat.npy'

X_test_node1 = path+'motifs/test_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/test_out_node2_3_0.0001/mat.npy'
X_test_window = path+'motifs/test_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/valid_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/valid_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/valid_out_btw_nodes_3_0.0001/mat.npy'

y_train = np.load(path+'labels/y_train_all.npy')
y_test = np.load(path+'labels/y_test_all.npy')
y_valid = np.load(path+'labels/y_valid_all.npy')

In [3]:
X_train=concat_motifs([X_train_node1, X_train_node2, X_train_window])
X_test=concat_motifs([X_test_node1, X_test_node2, X_test_window])
X_valid=concat_motifs([X_valid_node1, X_valid_node2, X_valid_window])

X_train_pairs = X_train.reshape(X_train.shape[0],X_train.shape[2]*X_train.shape[3])
X_valid_pairs = X_valid.reshape(X_valid.shape[0],X_valid.shape[2]*X_valid.shape[3])
X_test_pairs = X_test.reshape(X_test.shape[0],X_test.shape[2]*X_test.shape[3])

X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features_sampl_by_f(X_train_pairs, X_valid_pairs, X_test_pairs)
    

In [4]:
print X_train_normalized.shape

(26108, 5760)


In [6]:
print y_train.shape, y_valid.shape, y_test.shape

(26108, 17) (12220, 17) (7703, 17)


In [12]:
dnn_normalized = Genome3D_DNN_FC(num_features=5760, use_deep_CNN=False, num_tasks=17)
validation_data = (X_valid_normalized, y_valid)
dnn_normalized.train(X_train_normalized, y_train, validation_data)

Training model...
Train on 26108 samples, validate on 12220 samples
Epoch 1/100
26000/26108 [============================>.] - ETA: 0s - loss: 0.7113Epoch 0: validation loss: 1.061
Task 0: Balanced Accuracy: 50.55%	auROC: 0.519	 auPRC: 0.298
Positve Accuracy: 0.16%	 Negative Accuracy: 92.09%
Recall at 5% | 10% | 25% | 50% FDR: 9.0% | 0.2% | 0.2% | 2.3%	Num Positives: 1214.0	 Num Negatives: 3375.0

Task 1: Balanced Accuracy: 53.01%	auROC: 0.564	 auPRC: 0.362
Positve Accuracy: 0.00%	 Negative Accuracy: 89.63%
Recall at 5% | 10% | 25% | 50% FDR: 16.4% | 0.0% | 0.0% | 0.0%	Num Positives: 1717.0	 Num Negatives: 3508.0

Task 2: Balanced Accuracy: 54.28%	auROC: 0.559	 auPRC: 0.219
Positve Accuracy: 0.00%	 Negative Accuracy: 75.28%
Recall at 5% | 10% | 25% | 50% FDR: 33.3% | 0.0% | 0.0% | 0.0%	Num Positives: 1088.0	 Num Negatives: 4419.0

Task 3: Balanced Accuracy: 58.73%	auROC: 0.610	 auPRC: 0.356
Positve Accuracy: 0.00%	 Negative Accuracy: 69.23%
Recall at 5% | 10% | 25% | 50% FDR: 48.2% | 0

In [13]:
print(dnn_normalized.test(X_test_pairs, y_test))
print(dnn_normalized.test(X_train_pairs, y_train))
print(dnn_normalized.test(X_valid_pairs, y_valid))

Task 0: Balanced Accuracy: 50.06%	auROC: 0.502	 auPRC: 0.292
Positve Accuracy: 0.00%	 Negative Accuracy: 100.00%
Recall at 5% | 10% | 25% | 50% FDR: 0.1% | 0.0% | 0.0% | 0.0%	Num Positives: 640.0	 Num Negatives: 2350.0

Task 1: Balanced Accuracy: 54.97%	auROC: 0.581	 auPRC: 0.376
Positve Accuracy: 0.00%	 Negative Accuracy: 50.62%
Recall at 5% | 10% | 25% | 50% FDR: 59.3% | 0.0% | 0.0% | 0.7%	Num Positives: 1138.0	 Num Negatives: 2478.0

Task 2: Balanced Accuracy: 52.60%	auROC: 0.592	 auPRC: 0.261
Positve Accuracy: 0.00%	 Negative Accuracy: 24.03%
Recall at 5% | 10% | 25% | 50% FDR: 81.2% | 0.0% | 0.0% | 0.0%	Num Positives: 824.0	 Num Negatives: 3007.0

Task 3: Balanced Accuracy: 58.93%	auROC: 0.620	 auPRC: 0.406
Positve Accuracy: 0.00%	 Negative Accuracy: 68.58%
Recall at 5% | 10% | 25% | 50% FDR: 49.3% | 0.0% | 0.0% | 8.3%	Num Positives: 1238.0	 Num Negatives: 2685.0

Task 4: Balanced Accuracy: 52.58%	auROC: 0.548	 auPRC: 0.072
Positve Accuracy: 0.70%	 Negative Accuracy: 73.52%
Recall